In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import src.tokens as tokenization_utils
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

('2.1.2+cu121', '4.39.0.dev0', '12.1')

In [3]:
# from src.data.dataclasses import MultiCounterFactDataset

# dataset = MultiCounterFactDataset("../data")
# print(json.dumps(dataset[5].to_dict(), indent=2))

In [4]:
# from src.dataset.rome_dataclasses import CounterFactDataset

# # DATA_DIR = "../data"
# DATA_DIR = "../data/known/pythia-2.8b-deduped.json"

# counterfact = CounterFactDataset(
#     DATA_DIR,
#     absolute_path=True,
#     size=1000
# )

In [5]:
from src.models import ModelandTokenizer
from src.functional import filter_samples_by_model_knowledge

In [6]:
MODEL_PATH = "state-spaces/mamba-2.8b" # state-spaces/mamba-2.8b
# MODEL_PATH = "EleutherAI/pythia-2.8b-deduped"

mt = ModelandTokenizer(
    model_path=MODEL_PATH, 
    torch_dtype=torch.float32
)

2024-03-21 17:24:32 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-03-21 17:24:32 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b/resolve/main/config.json HTTP/1.1" 200 0
2024-03-21 17:24:43 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b/resolve/main/pytorch_model.bin HTTP/1.1" 302 0


/home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


2024-03-21 17:24:45 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-03-21 17:24:45 src.models INFO     loaded model <state-spaces/mamba-2.8b> | size: 10560.400 MB | dtype: torch.float32 | device: cuda


In [12]:
from src.dataset.dataclasses import load_relation

relation_names = [
        # "place_in_city",
        # "country_capital_city",
        # "person_occupation",
        # "person_plays_pro_sport",
        # "company_ceo",
        # "company_hq",
        "person_native_language",
        "landmark_in_country",
        "product_by_company",
    ]

DATA_DIR = "../data/relation/factual"

counter = {r:0 for r in relation_names}

for relation_name in relation_names:
    relation_file = os.path.join(DATA_DIR, f"{relation_name}.json")
    relation = load_relation(
        file=relation_file,
    )
    relation.select_icl_examples(0)
    relation = filter_samples_by_model_knowledge(
        mt = mt,
        relation = relation,
        limit=200
    )

    counter[relation_name] = len(relation)

counter

2024-03-21 17:34:35 src.dataset.dataclasses INFO     initialized relation -> "person native language" with 914 samples
2024-03-21 17:34:35 src.functional DEBUG    "person native language" | filtering with state-spaces/mamba-2.8b


2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Woodrow Wilson' -> answer='English' | predicted = ' English'(0.116) ==> (✓)
2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Pierre Bourdieu' -> answer='French' | predicted = ' French'(0.210) ==> (✓)
2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Michel Bouquet' -> answer='French' | predicted = ' French'(0.494) ==> (✓)
2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Nikolai Bukharin' -> answer='Russian' | predicted = ' the'(0.196) ==> (✗)
2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Jacques Dufilho' -> answer='French' | predicted = ' French'(0.464) ==> (✓)
2024-03-21 17:34:35 src.functional DEBUG    sample.subject='Shi Yongxin' -> answer='Chinese' | predicted = ' Chinese'(0.185) ==> (✓)
2024-03-21 17:34:36 src.functional DEBUG    sample.subject='Sergey Lavrov' -> answer='Russian' | predicted = ' Russian'(0.262) ==> (✓)
2024-03-21 17:34:36 src.functional DEBUG    sample.subject

{'person_native_language': 200,
 'landmark_in_country': 200,
 'product_by_company': 200}

In [10]:
relation.prompt_templates[0]

'{} speaks the language of'

In [12]:
counter

{'place_in_city': 103,
 'country_capital_city': 19,
 'person_occupation': 228,
 'person_plays_pro_sport': 221,
 'company_ceo': 91,
 'company_hq': 313,
 'person_native_language': 625,
 'landmark_in_country': 625,
 'product_by_company': 343}